In [ ]:
from cirq import Circuit, LineQid, unitary
import numpy as np
from qdit.gates import iSWAP, X, F, Rz, Z, Ph, S, Sdg, CX, CZ, Ry, X90, Id, Rx, C
from qdit.utils.math import unitary_equiv, unitary_to_special_unitary
from qdit.utils.sun_factorization import sun_factorization, su2_parameters
from qdit.compiling import compile_circuit

In [ ]:
d=2

c = Circuit()
q0 = LineQid(0,d)
q1 = LineQid(1,d)

# c.append(Ry(np.pi/2,d)(q0))
# c.append(iSWAP()(q0,q1))
# c.append(Ry(-np.pi,d)(q0))

# c.append(Ry(np.pi/2,d)(q1))
# c.append(Rz(-np.pi/2,d)(q1))
# c.append(iSWAP()(q0,q1))
# c.append(Ry(np.pi/2,d)(q0))
# c.append(iSWAP()(q0,q1))
# c.append(Ry(np.pi/2,d)(q0))
# c.append(iSWAP()(q0,q1))
# c.append(Rz(-np.pi/2,d)(q1))
# c.append(Ry(np.pi/2,d)(q1))


# # c.append(Rz(-np.pi/2,d)(q1))
# c.append(Ry(np.pi/2,d)(q1))
# c.append(Rz(-np.pi/2,d)(q1))
# c.append(iSWAP()(q0,q1))
# c.append(Ry(np.pi/2,d)(q0))
# c.append(iSWAP()(q0,q1))
# c.append(Ry(np.pi/2,d)(q0))
# c.append(iSWAP()(q0,q1))
# c.append(Rz(-np.pi/2,d)(q1))
# c.append(Ry(np.pi/2,d)(q1))
# c.append(iSWAP()(q0,q1))
# c.append(Rz(np.pi/2,d)(q1))
# c.append(Rz(-np.pi/2,d)(q0))


# print(c)
# print(np.round(unitary(c)/(0.5-0.5j),3))

c = Circuit()
c.append(CX(d)(q0,q1))
c.append(CX(d)(q1,q0))
print(c)
print(np.round(unitary(c),3))

comp_c = compile_circuit(c, d, native=True, native_gates='RzRyiSWAP')
print(comp_c)
print(np.round(unitary(comp_c),3))


[[1.+1.j]]
0 (d=2): ───CXc───CXt───
            │     │
1 (d=2): ───CXt───CXc───
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]
0 (d=2): ───Rz(0.50π)────────────────⨂───Rz(-0.50π)───Ry(0.50π)───Rz(0.50π)───⨂───Rz(-0.50π)───Ry(0.50π)───⨂────────────────────────────────────────⨂───Rz(-0.50π)───
                                     │                                        │                            │                                        │
1 (d=2): ───Rz(-0.50π)───Ry(0.50π)───⨂────────────────────────────────────────⨂────────────────────────────⨂───Rz(-0.50π)───Ry(0.50π)───Rz(0.50π)───⨂────────────────
[[ 0.-1.j -0.-0.j  0.+0.j  0.-0.j]
 [-0.-0.j  0.+0.j  0.-1.j  0.-0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -0.-1.j]
 [-0.+0.j -0.-1.j  0.-0.j  0.+0.j]]


In [ ]:
def iswap(a, q0, q1, d):
    out = [_ for _ in a] 
    out[q0] = (out[q0]-a[q0]-a[q1])%d
    out[q1] = (out[q1]-a[q0]-a[q1])%d
    out[q0+int(len(a)/2)] = (out[q0+int(len(a)/2)]-a[q0]-a[q1]+a[q0+int(len(a)/2)]+a[q1+int(len(a)/2)])%d
    out[q1+int(len(a)/2)] = (out[q1+int(len(a)/2)]-a[q0]-a[q1]+a[q0+int(len(a)/2)]+a[q1+int(len(a)/2)])%d
    
    return out

def c(a, i, d):
    out = [_ for _ in a]
    out[i], out[i+int(len(a)/2)] = (out[i]-out[i+int(len(a)/2)])%d, out[i]%d
    return out

def red_x(a, d):
    circuit = []
    out = [_ for _ in a]

    J = [j for j, x in enumerate(a[0:int(len(a)/2)]) if x!=0]
    while len(J)>1:
        # Process pairs of non-zero entries
        for i in range(len(J)):
            
            # For even indices that have a following entry
            if i%2==0 and i+1<len(J):
                target_x = out[J[i+1]]
                target_z = out[int(len(a)/2)+J[i+1]]
                control_x = out[J[i]]
                control_z = out[int(len(a)/2)+J[i]]

                while target_x!=0:
                    while control_x != 0 or control_z != 1:
                        out = c(out,J[i],d)
                        control_x = out[J[i]]
                        control_z = out[int(len(a)/2)+J[i]]
                        circuit.append(f'c{J[i]}')

                    while target_x != 1 or target_z != 1:
                        out = c(out,J[i+1],d)
                        target_x = out[J[i+1]]
                        target_z = out[int(len(a)/2)+J[i+1]]
                        circuit.append(f'c{J[i+1]}')

                    out = iswap(out,J[i],J[i+1],d)
                    target_x = out[J[i+1]]
                    target_z = out[int(len(a)/2)+J[i+1]]
                    control_x = out[J[i]]
                    control_z = out[int(len(a)/2)+J[i]]
                    circuit.append(f'iS{J[i],J[i+1]}')
                
                # Remove the cleared entry from our list
                J.remove(J[i+1])

    if J[0]!=0:
        if out[0]==0 and out[int(len(a)/2)] == 0:
            out = iswap(out,0,J[0],d)
            circuit.append(f'iS{0,J[0]}')
        
        while out[0]!=0 or out[int(len(a)/2)]!=1:
            out = c(out,0,d)
            circuit.append('c0')

        while out[J[0]]!=1 or out[int(len(a)/2)+J[0]]!=1:
            out = c(out,J[0],d)
            circuit.append(f'c{J[0]}')

        out = iswap(out,0,J[0],d)

    return circuit, out

a = [1,0,0,0,0,0]
# print(c(a,0,2))
print(iswap(a,0,1,2))
# b = [0,0,0,1,0,0]
# d=2

# from qdit.utils.math import check_pauli_relation
# print(check_pauli_relation(a, b, d=2, non_commute=1))

# # print(iswap(a, 0, 1, 3))
# circ, out = red_x(a,d=2)
# print(out)
# print(circ)

[0, 1, 0, 1, 1, 0]


In [ ]:
print(circ)

for gate in circ:
    if gate == 'iS':
        b = iswap(b,0,1,d)
    elif gate =='c0':
        b = c(b,0,d)
    else: 
        b = c(b,1,d)
print(b)

['iS', 'c0', 'c1', 'c1', 'iS']
[1, 0, 1, 1]
